In [1]:
import numpy as np
import math

## Reference Data

## Run Data

In [2]:
def better_mask_for(a, b):
    sa = np.sum(a, axis=-1)
    sb = np.sum(b, axis=-1)
    return sa > sb

def max_diff_for(a, b):
    sa = np.sum(a, axis=-1)
    sb = np.sum(b, axis=-1)
    
    diff = sa-sb
    min_diff = np.min(diff)
    max_diff = np.max(diff)
    
    min_pos = np.amin(diff)
    max_pos = np.amax(diff)
    
    print("min", min_diff, "max", max_diff)
    
    return diff.astype("float"), min_diff, max_diff, min_pos, max_pos

# which of the above graph's points protrude further
def maximum_qs(qs_list, graph_list=None):
    best = np.zeros_like(qs_list[0])
    
    if graph_list is None:
        for qs in qs_list:
            better_mask = better_mask_for(qs, best)
            best[better_mask] = qs[better_mask]

        return best
    else:
        best_graphs = np.zeros_like(graph_list[0]) - 1
        best_file_idx = np.zeros_like(graph_list[0], dtype=int) - 1
        
        for qs, gr, i in zip(qs_list, graph_list, range(len(graph_list))):
            better_mask = better_mask_for(qs, best)
        
            best[better_mask] = qs[better_mask]
            best_graphs[better_mask] = gr[better_mask]
            best_file_idx[better_mask] = i
    
        return best, best_graphs, best_file_idx

In [3]:
import glob, re
import itertools, operator

### All Graphs

In [4]:
files = [(fn, int(re.findall(r"kSys(\d+)", fn)[0])) for fn in glob.glob("./volumes/all-graphs/*.npz")]
all_graph_data = {}
all_graph_files = {}
all_graph_id = {}
all_graph_file = {}
all_graph_ctr = {}

for (fn, kSys) in files:
    if not kSys in all_graph_data:
        all_graph_data[kSys] = []
        all_graph_id[kSys] = []
        all_graph_files[kSys] = []
        all_graph_ctr[kSys] = 0
    npz = np.load(fn)
    all_graph_data[kSys].append(npz["qs"])
    all_graph_id[kSys].append(npz["graph"])
    all_graph_files[kSys].append(fn)
    all_graph_ctr[kSys] += 1
    
for key in all_graph_data:
    all_graph_data[key], all_graph_id[key], all_graph_file[key] = maximum_qs(all_graph_data[key], all_graph_id[key])
    print(key, all_graph_ctr[key])

6 30
5 21
4 6
2 1
3 2


In [5]:
import tarfile

_load_adjm_list_cache = {}
def load_adjm_list(fn):
    if fn in _load_adjm_list_cache:
        return _load_adjm_list_cache[fn]
    
    l = [None]
    with tarfile.open(fn, mode="r") as archive:
        for file_meta in archive:
            if file_meta.type != tarfile.REGTYPE:
                continue
            
            adjm_matches = re.findall(r"([0-1]+)\.json\.gz", file_meta.name)
            assert len(adjm_matches) == 1, "cannot find adjm"
            
            KTOT = math.sqrt(len(adjm_matches[0]))
            assert KTOT.is_integer(), "adjacency matrix not square"
            KTOT = int(KTOT)
            if KTOT > 8:
                continue
            
            l.append(adjm_matches[0])
            
    _load_adjm_list_cache[fn] = l
    return l
            

def best_graph_for(kSys, file_idx, entry_idx):
    fn = all_graph_files[kSys][file_idx]
    tag = re.findall(r"all-graphs-([^-]+)", fn)[0]
    fn = glob.glob(f"./results/all-graphs/{tag}/*{tag}*.lambdas.tar.gz")[0]
    
    return load_adjm_list(fn)[entry_idx]
    

all_graph_best_swatch_lut_ = {}
for k in [2,3,4,5]:
    all_graph_best_swatch_lut_[k] = {
        pair: best_graph_for(k, *pair) for pair in set(zip(all_graph_file[k], all_graph_id[k]))
    }

In [12]:
print("{" + ", ".join([f'"{v}"' for v in all_graph_best_swatch_lut_[5].values()]) + "}")

{"0000111100001111000011110000111111110001111100001111000011111000", "001101000111100011110001011001111110", "000010000010000010000010111101000010", "0001100000001001000010101000100011110101000010000010000001001000", "0011111100111000110110001110100011110000100000001000000010000000", "0000100100001001000010010000111111110011000100000001100011111000", "001110000010100110101010111101000010", "0001111100001000000010001000000011100000100000001000000010000000", "0011100100011011100011101100000111100011001000000110100011011000", "011111101111110111111011111101111110", "001010000110100010010010111101000010", "0001101100011011000110111110011111100111000110001111100011111000", "0011111100111011110110011110100111110001100000001100000011111000", "000110000110000110111001111000000100", "0011000100011000100011111100000001100000001000000010000010100000", "000111000111000111111000111000111000", "0011100000011111100110001110100011110000010000000100000001000000", "00111000000011111001100010101000111100

In [13]:
# lut's from MM for graph isomorphism mapping
graph_isom_mapping = {}
graph_isom_mapping[2] = {"0111101111001100": "0011001111011110", "011100100": "001001110", "0101101101001100": "0001001101011110"}
graph_isom_mapping[3] = {"001001001111110111011000011000111000": "000101000011000011100011011101111110", "0011001111011110": "0011001111011110", "0111101111011110": "0111101111011110", "0010001011010010": "0001000100011110", "0011100111110001100011000": "0001100011000111110011100", "001011001111110101011000110000111000": "000011000101000111011010101101111010", "001111001111110001110000110000111000": "000011000011000111001011111100111100", "0111110100110001000010000": "0000100001000110010111110", "001111001011110000100000110000110000": "000001000011000011000011011100111100"}
graph_isom_mapping[4] = {"0011111100111111110000011100000111000000110000001100000011110000": "0000001100000011000000110000011100000111000110111111110011111100", "0001000000010000000100001110111100010000000100000001000000010000": "0000000100000001000000010000000100000001000000010000000111111110", "0001011100011001000110101110100001110000100000001010000011000000": "0000000100001001000001010000111001010010001100100001110111100010", "0011000000010111100100001110101100010000010000000101000001010000": "0000001000000001000100010010000100000011000000111000110101111110", "0111111110111111110111111110111111110000111100001111000011110000": "0000111100001111000011110000111111110111111110111111110111111110", "0001111100011111000111111110111111110000111100001111000011110000": "0000111100001111000011110000111111110001111100011111000111111110", "0111000101100011010001110111000100000010000001000": "0000001000000100000010000111000101100011011111110", "0001111000111100011111110000111000011100001110000": "0000111000011100001110000111111100011110001111000", "001111001111110011110011111100111100": "011110101101110011110011101101011110", "0001000011000101110001000": "0001000001000011000101110", "000111000111000111111001111000111100": "000111000111000111111000111001111010", "0001111100010000000100001110001110000000100000001001000010010000": "0000001000000010000000010000000100000011000000111100110100111110", "0011000000110111110010011100000000100000010000000100000001100000": "0000001000000001000000010000101000010001000000111001010101101110", "011111101111110101111011110100111100": "000111001111010111111011111101111110", "0001001100011101000111111110000101100000011000001010000011110000": "0000100100001001000100010010011011000110000110110001110111100110", "001100000111100011110000011000011000": "000011000011000110001001111000110100", "0011000000110000110100001110111100010000000100000001000000010000": "0000000100000001000000010000000100000011000000110000110111111110", "001100000111100100111000010000010000": "000001000001000110001010001101110010", "0011000000010000100011111100000000100000001000000010000000100000": "0000010000000001000000010000000100000001100000100000010101111010"}
graph_isom_mapping[5] = {"0000111100001111000011110000111111110001111100001111000011111000": "0000111100001111000011110000111111110000111100001111000111110010", "001101000111100011110001011001111110": "011001100101100011010011001101111110", "000010000010000010000010111101000010": "000001000001000001000001000001111110", "0001100000001001000010101000100011110101000010000010000001001000": "0000001000000001000100010010000100000101000010011000000101111110", "0011111100111000110110001110100011110000100000001000000010000000": "0000000100000001000000010000111000010111000110110001110111101110", "0000100100001001000010010000111111110011000100000001100011111000": "0000010000000101000000110000001100000011110000110011110101111110", "001110000010100110101010111101000010": "000001000001000111001011001101111110", "0001111100001000000010001000000011100000100000001000000010000000": "0000001000000010000000010000000100000001000000011100000100111110", "0011100100011011100011101100000111100011001000000110100011011000": "0000001000011100000001110100110101010011011100011010100100111110", "011111101111110111111011111101111110": "011111101111110111111011111101111110", "001010000110100010010010111101000010": "000001001001010001000011000101111110", "0001101100011011000110111110011111100111000110001111100011111000": "0000001100001111000011110000111101110011011100111111110011111100", "0011111100111011110110011110100111110001100000001100000011111000": "0000000100100001010111100010111100110111001110110011110111011110", "000110000110000110111001111000000100": "000001000011000011000011011100111100", "0011000100011000100011111100000001100000001000000010000010100000": "0000000100000001000110000010001000100001000000110001010111001110", "000111000111000111111000111000111000": "001110001110110001110001110001001110", "0011100000011111100110001110100011110000010000000100000001000000": "0000000100000001000000010000111000010110000110110001110111100110", "0011100000001111100110001010100011110001010000000100000001001000": "0000001000000010000000110000110100010101000110011110000100111110", "0001111100011000000010001100000011100000100000001000000010000000": "0000001000000001000000010000000100000110000010011000100101110110", "0001101100001000000010001000110111110001000100001000000010011000": "0000010000000010000000010000000100000111100010110100110100111110", "0001111100010000000010001100100010110000100000001000000010000000": "0000010000000010000000010000000100000001100000110100010100111110", "0000111100001111000011110000111111110000111100001111000011110000": "0001111000011110000111101110000111100001111000011110000100011110", "0011000000111110001001100000011000001000000100000": "0000001000000100011000010010001000100010011100110", "000011000011000011000011111101111110": "000011000011000011000011111101111110", "0001111100001000000010001000000111100000100000001000000010010000": "0000001000000010000000010000000100000101000010011100000100111110", "0001111100010001000010011100111010110111100110001001100011101000": "0000110000001001000001110000011111000011101100110011110101111110", "000110000110000110111010111101000010": "000001000011000011000011011101111110"}

In [14]:
all_graph_best_swatch = {}
all_graph_best_swatch_lut = {}
for k in all_graph_best_swatch_lut_:
    isom_graph_to_swatch = set(graph_isom_mapping[k].values())
    isom_graph_to_swatch = dict(zip(isom_graph_to_swatch, range(len(isom_graph_to_swatch))))
    print(k, isom_graph_to_swatch)
    
    pair_to_swatch = {
        pair: isom_graph_to_swatch[
            graph_isom_mapping[k][
                all_graph_best_swatch_lut_[k][pair]
            ]
        ]
        for pair in all_graph_best_swatch_lut_[k]
    }
    print(k, pair_to_swatch)
    
    all_graph_best_swatch[k] = np.array([
        pair_to_swatch[pair] for pair in zip(all_graph_file[k], all_graph_id[k])
    ])
    
    # reverse lookup table to be from swatch > graph
    all_graph_best_swatch_lut[k] = dict(
        (pair_to_swatch[pair], graph_isom_mapping[k][
            all_graph_best_swatch_lut_[k][pair]
        ])
        for pair in pair_to_swatch
    )
    
    print(k, all_graph_best_swatch_lut[k])
    

2 {'001001110': 0, '0011001111011110': 1, '0001001101011110': 2}
2 {(0, 1): 1, (0, 3): 0, (0, 5): 2}
2 {1: '0011001111011110', 0: '001001110', 2: '0001001101011110'}
3 {'000101000011000011100011011101111110': 0, '0001000100011110': 1, '0001100011000111110011100': 2, '000011000011000111001011111100111100': 3, '0111101111011110': 4, '0000100001000110010111110': 5, '0011001111011110': 6, '000001000011000011000011011100111100': 7, '000011000101000111011010101101111010': 8}
3 {(0, 1): 0, (0, 15): 6, (1, 87): 4, (0, 25): 1, (0, 12): 2, (0, 5): 8, (0, 3): 3, (1, 11): 5, (0, 23): 7}
3 {0: '000101000011000011100011011101111110', 6: '0011001111011110', 4: '0111101111011110', 1: '0001000100011110', 2: '0001100011000111110011100', 8: '000011000101000111011010101101111010', 3: '000011000011000111001011111100111100', 5: '0000100001000110010111110', 7: '000001000011000011000011011100111100'}
4 {'011110101101110011110011101101011110': 0, '000000010000100100000101000011100101001000110010000111011110001

In [55]:
all_graph_best_swatch_orig_lut = {}
for k in all_graph_best_swatch_lut:
    qs_swatch_lut = all_graph_best_swatch_lut[k]
    
    orig_lut = {}
    
    for s in qs_swatch_lut:
        adjm = qs_swatch_lut[s]
        for orig in graph_isom_mapping[k]:
            if graph_isom_mapping[k][orig] == adjm:
                orig_lut[s] = orig
                break
        
    all_graph_best_swatch_orig_lut[k] = orig_lut
    assert len(orig_lut) == len(all_graph_best_swatch_lut[k]), "map could not be inverted"

In [73]:
qs_comp = all_graph_data[5]
qs_swatch = all_graph_best_swatch[5]
qs_swatch_lut = all_graph_best_swatch_orig_lut[5]
x,y,z = qs_comp.transpose()

## Rep Codes

In [210]:
files = [(fn, int(re.findall(r"-in-(\d+)", fn)[0])) for fn in glob.glob("./volumes/concat_codes/*1-in-*.npz")]
rep_graph_data = []
rep_graph_swatch = []
for (fn, kk) in files:
    if int(kk) == 1: continue
    npz = np.load(fn)
    rep_graph_data.append(npz["qs"])
    rep_graph_swatch.append(npz["graph"]*0 + kk)
rep_graph_data, rep_graph_swatch, _ = maximum_qs(rep_graph_data, rep_graph_swatch)

In [16]:
qs_comp = rep_graph_data
qs_swatch = rep_graph_swatch
x,y,z = qs_comp.transpose()

## Concat Codes

In [209]:
cat_graph_files = sorted(glob.glob("./volumes/concat_codes/*[2|3|4|5|6]-in-*.npz"), key=lambda s: tuple([
    int(k) for k in re.findall(r"(\d+)-in-(\d+)", s)[0]
]))
cat_graph_data = []
cat_graph_swatch = []
for i, fn in enumerate(cat_graph_files):
    npz = np.load(fn)
    cat_graph_data.append(npz["qs"])
    cat_graph_swatch.append(npz["graph"]*0 + i)
    
cat_graph_data, cat_graph_swatch, _ = maximum_qs(cat_graph_data, cat_graph_swatch)

In [198]:
cat_graph_files

['./volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6RGdAaUAK.2-in-2.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6RmdAaUBxQH5+Cg==.2-in-3.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6SGM-S2dAaD9MZ0AK.2-in-4.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6SmM-S2dAaD9MZ0BwP05+fgo=.2-in-5.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6TGM-S2dAaD9MZ0BwP01nQAo=.2-in-6.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6TmM-S2dAaD9MZ0BwP01nQHg-Tn5+Cg==.2-in-7.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6RmtASXpfUX5+Cg==.3-in-2.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6SWU-R2hyX0NUak9BSn5+fgo=.3-in-3.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6TGU-R2hyX0NUak9BSk1XQER+fn5+Cg==.3-in-4.json.gz.tar-best.npz',
 './volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw

## 2 Level Codes

In [208]:
lvl_graph_files = sorted(glob.glob("./volumes/special_felix/*.npz"), key=lambda s: int(re.findall(r"kSys(\d+)", s)[0]))
lvl_graph_data = []
lvl_graph_swatch = []
for i, fn in enumerate(lvl_graph_files):
    npz = np.load(fn)
    lvl_graph_data.append(npz["qs"])
    lvl_graph_swatch.append(npz["graph"]*0 + i)
    
lvl_graph_data_all, lvl_graph_swatch_all, _ = maximum_qs(lvl_graph_data, lvl_graph_swatch)

In [170]:
qs_comp = lvl_graph_data_all
qs_swatch = lvl_graph_swatch_all
qs_swatch_lut = [ re.findall(r"graphcodes\.(.+)-kSys", f)[0] for f in lvl_graph_files ]
x,y,z = qs_comp.transpose()

In [164]:
qs_comp = lvl_graph_data[5]
print(lvl_graph_files[5])
x,y,z = qs_comp.transpose()

./volumes/special_felix/graphcodes.Pj5zcGFyc2U2PDw6U19gYF9jY19mZl9paV9sbF9fX19+fn5+fgo=-kSys19-kTot20.json.gz.tar-best.npz


## 2 Level vs Cat vs Rep

In [211]:
qs_comp, qs_swatch, _ = maximum_qs([
    lvl_graph_data_all,
    rep_graph_data,
    cat_graph_data], [
    lvl_graph_swatch_all*0 + 0,
    rep_graph_swatch*0 + 1, 
    cat_graph_swatch*0 + 2]
)
x,y,z = qs_comp.transpose()

## Single Code

In [19]:
f = glob.glob("./volumes/special_felix/*kSys19-kTot20*.npz")[0]
print(f)
data_comp = np.load(f)
qs_comp = data_comp["qs"]
x,y,z = qs_comp.transpose()

./volumes/special_felix/graphcodes.Pj5zcGFyc2U2PDw6U19gYF9jY19mZl9paV9sbF9fX19+fn5+fgo=-kSys19-kTot20.json.gz.tar-best.npz


## Plot

In [16]:
import mayavi
import mayavi.mlab as mlab

from scipy import spatial

In [227]:
from matplotlib import pyplot

# nonlinear color transformation around midpoint
from matplotlib.colors import SymLogNorm
color_mapper_diff = SymLogNorm(linthresh=.00008, linscale=.00008, vmin=-.1, vmax=.1)
color_mapper_swatch = lambda x: x

from matplotlib import cm

#### Reference Graph

In [160]:
data_ref = np.load("./volumes/single-letter-best.npz")
qs_ref = data_ref["qs"]

In [121]:
f = glob.glob("./volumes/concat_codes/*5-in-5.*-best.npz")[0]
print(f)
data_ref = np.load(f)
qs_ref = data_ref["qs"]

./volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6WWQ-QEFCQ2RkZGRfQEFCQ2lpaWlfQEFCQ25ubm5fQEFCQ3Nzc3NfQEFCQwo=.5-in-5.json.gz.tar-best.npz


#### Plot Type

In [228]:
# diff plot
diff, min_diff, max_diff, min_pos, max_pos = max_diff_for(qs_comp, qs_ref)

dxx, dyy, dzz, dds = [ a.reshape(int(np.sqrt(len(x))), -1) for a in (x, y, z, diff) ]
colorspace = np.linspace(-.1, .1, 255)
viridis = cm.get_cmap('Spectral')
INTERPOLATE = True
color_mapper = color_mapper_diff

min -0.00594720111735389 max 0.026736971693860356


In [225]:
# swatch plot
dxx, dyy, dzz, dds = [ a.reshape(int(np.sqrt(len(x))), -1) for a in (x, y, z, qs_swatch/(max(qs_swatch)+1)) ]
colorspace = np.linspace(0, 1, 255)
viridis = cm.get_cmap('hsv')
INTERPOLATE = False
color_mapper = color_mapper_swatch

In [ ]:
viridis(color_mapper(colorspace))

In [214]:
ANTIALIAS = 8
LINEWIDTH = 3

mlab.options.offscreen = True
mlab.figure(bgcolor=(1,1,1), size=(1024,1024))

mesh = mlab.mesh(dxx, dyy, dzz, scalars=dds, colormap="Spectral", opacity=.95)
mesh.actor.property.lighting = False
mesh.actor.mapper.interpolate_scalars_before_mapping = INTERPOLATE
mesh.module_manager.scalar_lut_manager.lut.table = 255*viridis(color_mapper(colorspace))
mesh.module_manager.scalar_lut_manager.data_range = (min(colorspace), max(colorspace))

AXES_RANGE = .52
def draw_line(px, py, pz, color=(0,0,0)):
    mlab.plot3d(px, py, pz, line_width=LINEWIDTH*ANTIALIAS, tube_radius=None, color=color)
    
draw_line((0,AXES_RANGE),(0,0),(0,0))
draw_line((0,0),(0,AXES_RANGE),(0,0))
draw_line((0,0),(0,0),(0,AXES_RANGE))

for i in np.arange(.1, .4, .1):
    #bottom
    draw_line((i,i),(0,AXES_RANGE),(0,0), color=(.75,.75,.75))
    draw_line((0,AXES_RANGE),(i,i),(0,0), color=(.75,.75,.75))
    #left
    draw_line((i,i),(0,0),(0,AXES_RANGE), color=(.75,.75,.75))
    draw_line((0,AXES_RANGE),(0,0),(i,i), color=(.75,.75,.75))
    #right
    draw_line((0,0),(i,i),(0,AXES_RANGE), color=(.75,.75,.75))
    draw_line((0,0),(0,AXES_RANGE),(i,i), color=(.75,.75,.75))


draw_line((.5,.5),(0,AXES_RANGE),(0,0), color=(.5,.5,.5))
draw_line((0,AXES_RANGE),(.5,.5),(0,0), color=(.5,.5,.5))

draw_line((.5,.5),(0,0),(0,AXES_RANGE), color=(.5,.5,.5))
draw_line((0,AXES_RANGE),(0,0),(.5,.5), color=(.5,.5,.5))

draw_line((0,0),(.5,.5),(0,AXES_RANGE), color=(.5,.5,.5))
draw_line((0,0),(0,AXES_RANGE),(.5,.5), color=(.5,.5,.5))

# reference
if True:
    ps = np.arange(0,AXES_RANGE/3,.001)
    
    mesh = spatial.KDTree(qs_comp)
    def min_to_surface(xs, ys, zs):
        pts = np.array([xs,ys,zs]).transpose()
        dists = [
            mesh.query(p)[0] for p in pts
        ]
        i = np.argsort(dists)[:2]
        pa = pts[i[0]]
        pb = pts[i[1]]
        
        if np.linalg.norm(pa) > np.linalg.norm(pb):
            return pa.transpose()
        return pb.transpose()
        
    def draw_cross(xs, ys, zs):
        x, y, z = min_to_surface(xs, ys, zs)
        mlab.points3d(x, y, z, scale_factor=.007*(LINEWIDTH**.25), reset_zoom=False, color=(0,0,0), resolution=20)
        
    #depolarizing
    draw_cross(ps, ps, ps)
    #2-pauli
    draw_cross(ps, 0.*ps, ps)
    #bb84
    draw_cross(ps-ps**2, ps**2, ps-ps**2)

#colorbar = mlab.colorbar(orientation='vertical', nb_labels=max(qs_swatch)+1)
#colorbar.label_text_property.font_family = 'courier'
#colorbar.label_text_property.bold = 0
#colorbar.label_text_property.font_size=24
#colorbar.label_text_property.color = (0,0,0)

scene = mlab.gcf().scene
scene.parallel_projection = True

In [215]:
mlab.savefig("/home/jkrb2/Desktop/snapshot.png", magnification=ANTIALIAS)

In [172]:
# mathematica exportable graph list for swatch plot
mmstr = []
for swatch in set(qs_swatch):
    r, g, b, a = viridis(color_mapper(swatch/(max(qs_swatch)+1)))
    mmstr.append(f"{{ RGBColor[{round(r, 4)},{round(g, 4)},{round(b, 4)}], \"{qs_swatch_lut[swatch]}\" }}")
print("{" + ",".join(mmstr) + "}")

{{ RGBColor[1.0,0.0,0.0], "Pj5zcGFyc2U2PDw6TWBFS0lTYF17R09gfn5+fgo=" },{ RGBColor[0.9882,0.961,0.0], "Pj5zcGFyc2U2PDw6T2BFS0lTYF17TFlgQUZ+fgo=" },{ RGBColor[0.0312,1.0,0.0], "Pj5zcGFyc2U2PDw6UF9gYF9jY19mZl9paV9fX19+fgo=" },{ RGBColor[0.0,1.0,0.9647], "Pj5zcGFyc2U2PDw6UV9gYF9jY19mZl9paV9sbF9ffgo=" },{ RGBColor[0.0,0.0625,1.0], "Pj5zcGFyc2U2PDw6Ul9gYF9jY19mZl9paV9sbF9fXwo=" },{ RGBColor[0.9335,0.0,1.0], "Pj5zcGFyc2U2PDw6U19gYF9jY19mZl9paV9sbF9fX19+fn5+fgo=" }}


In [33]:
max(qs_swatch)

2